# BC205: Algorithms for Bioinformatics.
## VI. Rapid Searches

### Christoforos Nikolaou

### Dealing with the problem of sequence similarity and comparison, we have up to now seen how:
* We can quantify the similarity between two sequences using scoring schemes that are either arbitrarily defined, or based on **substitution matrices** obtained from molecular evolution approaches.
* We can define an objective methodology that maximizes sequence similarity through the concept of **alignment**.
* We can distinguish between **global** (for the entire sequence length) and **local** (for subparts of the sequences) alignment.

### We are now moving on into a different aspect of the problem of sequence similarity that stems from the scale of sequence size and volume

a. Creating of a PWM from a given sequnce

In [13]:
# Code here
import regex as re
f=open("files/gata.fa", 'r')
seqs = []
for line in f:
    x=re.match(">", line)
    if x == None:
        seqs.append(line.rstrip())
#
def pwm(sequences):
    nuc = ['A', 'C', 'G', 'T']
    profile=[[0 for i in range(len(sequences[0]))] for j in range(len(nuc))]
    #
    for instance in sequences:
        for j in range(len(instance)):
            residue=instance[j]
            profile[nuc.index(residue)][j]+=1
            profile[nuc.index(residue)][j]=float(profile[nuc.index(residue)][j])
    import numpy as np
    pwm = np.array(profile)
    pwm = pwm/len(sequences)
    return(pwm)

mypwm=pwm(seqs)
    


KMP

In [14]:
def readfasta(fastafile):
    import regex as re
    f=open(fastafile, 'r')
    seq = ""
    total = 0
    for line in f:
        x=re.match(">", line)
        if x == None:
            length=len(line)
            total=total+length
            seq=seq+line[0:length-1]
    seq=seq.replace('N','')
    f.close()
    return(seq)

targetsequence=readfasta("files/ecoli.fa")


BM

In [11]:

def nuccomp(sequence):
    import numpy as np
    nucfreq = [0, 0, 0, 0]
    nuc = ['A', 'C', 'G', 'T']
    for i in range(len(nuc)):
        nucfreq[i]=sequence.count(nuc[i])
    nucfreq=np.array(nucfreq)/len(sequence)
    return(nucfreq)

nucfreqs=nuccomp(targetsequence)

FASTA

FastA - Steps
Hashing: FastA locates regions of the query sequence and matching regions in the database sequences that have high densities of exact word matches. (without gaps) The length of the matched word is called the ktup parameter.
Scoring: The ten highest scoring regions are rescored using a scoring matrix. The score for such a pair of regions is saved as the init1 score.
Introduction of Gaps: FastA determines if any of the initial regions from different diagonals may be joined together to form an approximate alignment with gaps. Only non-overlapping regions may be joined. The score for the joined regions is the sum of the scores of the initial regions minus a joining penalty for each gap. The score of the highest scoring region, at the end of this step, is saved as the initn score.
Alignment: After computing the initial scores, FastA determines the best segment of similarity between the query sequence and the search set sequence, using a variation of the Smith-Waterman algorithm. The score for this alignment is the opt score.
Random Sequence Simulation: In order to evaluate the significance of such alignment FastA empirically estimates the score distribution from the alignment of many random pairs of sequences. More precisely, the characters of the query sequences are reshuffled (to maintain bias due to length and character composition) and searched against a random subset of the database. This empirical distribution is extrapolated, assuming it is an extreme value distribution, and each alignment to the real query is assigned a Z-score and an E-score.

In [15]:
# Creation of a PSSM
def pssm(pwm, nucfreqs):
    import numpy as np
    import math
    pseudocount=0.01
    pssm=[[0 for i in range(len(pwm[0]))] for j in range(len(nucfreqs))]
    for i in range(len(nucfreqs)):
        pssm[i]=(np.array(pwm[i])+pseudocount)/nucfreqs[i]
    for i in range(len(pssm)):
        for k in range(len(pssm[0])):
            pssm[i][k]=math.log(pssm[i][k])/math.log(2)
    return(pssm)

mypssm=pssm(mypwm, nucfreqs)

In [ ]:
c. Searching a sequence with a PWM/PSSM or with Hamming Distance 

In [16]:
def pssmSearch(pssm, sequence, threshold):
    nuc = ['A', 'C', 'G', 'T']
    hits = []
    instances = []
    for i in range(len(sequence)-len(pssm[0])):
        instance=sequence[i:i+len(pssm[0])]
        score=0
        for l in range(len(instance)):
            score=score+pssm[nuc.index(instance[l])][l]
        if (score > threshold):
            hits.append(i)
            instances.append(instance) 
    return(hits, instances)

out=pssmSearch(mypssm, targetsequence, 9)

BLAST

In [ ]:
# code to be added

### The next problem. Discover a new motif from a given set of sequences

#### Part 1. Formulating the problem
1. Given a set of sequences that each contains an instance of the motif, find the motif.

#### A first approach 

Assuming we have a way to 


In [ ]:
# code here

1. Given a set of s sequences: Find a set of k-mers (for a given
length k, one from each sequence) that maximizes the score (or
minimizes the distance) of each (one) k-mer with its sequence
2. Collect k-mers
3. Create a motif from them


In [ ]:
# code

### Brute Force Approach

What is the complexity of the BFA?
1. Number of k-mers 4k
2. Number of k-mers in each sequence: (n − k + 1)
3. Number of calculations for each k-mer given s sequences of
length n: (n − k + 1)s
4. Total number of calculations 4k (n − k + 1)s

The complexity of the algorithm is at least O(ns ).

We need something faster!


In [ ]:
# code

* Assuming we have a way to calculate the distance of a k-mer k

```
from a given sequence seq  
    for k in kmers:  
        for seq in sequences:  
            if distance(k, seq)<min_distance:  
                min_distance<-distance(k,seq)  
                motif[seq]<-k
```

* Because each k-mer needs to pass only once through each
sequence, the median string has O(4k ) complexity because k is
(usually) much shorter than the length of the sequence.

* However, it is still quite slow and for k>10 its implementation
is still unapplicable.


In [ ]:
# code

### An alternative

* Assume a greedy approach to go through all sequences
updating a motif every time

* Starting from sequence i:

1. find the most common k-mer
2. create a profile from it (adding pseudocounts to all 0-values)
3. go to the next sequence
4. choose the k-mer that best fits the profile
5. store that k-mer in the collection and update profile
6. iterate steps 3->5.

* We’ ve just described a Greedy approach for discovering a
motif p of a given length k among t sequences.


In [ ]:
# code

### A greedy Approach

* Assuming a set of s sequences and a given consensus k-mer k:

* We will construct a PWM “on the go” as we move from one
sequence to the next.
1. For i=1 :
2. For each k in seq_i:  
    2.1 For i = 2 to i = s:  
    2.2 Find the best (smallest distance) kmer in seq_i  
    2.3 Build a profile  
    2.4 If the score(profile) is better than all previous update profile  
3. Repeat


### Analysis. Greedy Approach

1. Why Greedy: It takes kmers from the first sequence only to
scan in the following. Thus it doesn’t go through all
combinations of sequences and k-mers. As we’ve seen above
the trade-off is speed.
2. KEY: It assumes that all sequences contain the motif. If the
first sequence doesn’t contain the motif (in any variation) then
we are doomed in looking for something that is non-sensical.
3. A way to go around this is to sample a small percentage of
sequences randomly, which brings us to the next-to-last chapter
of the motif finding problem


### A randomized approach

* In the Greedy Approach we take the kmers from the first
sequence and scan over the rest. In this way an initial wrong
choice may lead you to disastrous results.
* In a Randomized Approach we start, instead with a
collection of s k-mers, one from each sequence, build a profile,
scan the sequences with that profile, update it and repeat until
the k-mer set is good enough match for the updated profile.
* Stop and think of the problems we get rid of with this
approach.


### Pseudocode

```
for seq in sequences:  
    profile[seq]<-random(k, seq)  
    while distance(profile, sequences)>threshold  
        for seq in sequences:  
        profile[seq]<-max(k, profile, seq)  
```

In [ ]:
# code